In [1]:
from collections import defaultdict
from bs4 import BeautifulSoup, Tag
import pandas as pd
import requests
import re

BASE_URL = 'http://www.columbia.edu'

In [2]:
def subjUrl(letter):
    return BASE_URL + '/cu/bulletin/uwb/sel/subj-{}.html'.format(letter)

def getSubjects(letter):
    r = requests.get(subjUrl(letter))
    r.raise_for_status()
    soup = BeautifulSoup(r.text, 'lxml')
    table = soup.table
    rows = table.find_all('tr')[3:-2]
    return rows

In [3]:
paths = []
for c in range(65, 91):
    rows = getSubjects(chr(c))
    for row in rows:
        href = row.a['href']
        if 'subj/AU' in href: # auditing
            break
        if '__' in href: # seemingly invalid subjects
            continue
        paths.append(href)

In [63]:
subjPattern = re.compile('subj\/(\w{4})')

def getSubjectFromPath(p):
    matches = subjPattern.findall(p)
    if len(matches) == 0:
        return None
    return matches[0]    

In [71]:
def parseForSection(contents):   
    components = contents[2].contents 
    section = {
         'number': contents[0].a.string.replace('Section ', ''),
    }
    header = None
    for component in components:
        if header is not None:
            text = component.strip()
            try:
                text = int(text)
            except ValueError:
                pass # This is expected since not all values are integers (e.g. instructor names)
            section[header] = text
            header = None
        if isinstance(component, Tag) and component.name == 'b':
            header = component.string.replace(':', '').strip()
    return section

def getCoursesFromPath(p):
    r = requests.get(BASE_URL + p)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, 'lxml')
    subject = getSubjectFromPath(p)
    course = None
    courses = []
    for tr in soup.table.find_all('tr')[2:-1]:
        if tr.td.get('colspan') is not None:        
            course = tr.td.b.contents[-1]
            continue
        section = parseForSection(tr.contents)
        section['course'] = course
        section['subject'] = subject
        courses.append(section)
    return courses

In [72]:
courses = []
for p in paths:    
    courses += getCoursesFromPath(p)
    print(f'Scanned courses for subject {getSubjectFromPath(p)}.')

Scanned courses for subject ACCT.
Scanned courses for subject ACTU.
Scanned courses for subject ADDN.
Scanned courses for subject AFCV.
Scanned courses for subject AFAS.
Scanned courses for subject AFRS.
Scanned courses for subject AKAD.
Scanned courses for subject AMST.
Scanned courses for subject ANCS.
Scanned courses for subject ANES.
Scanned courses for subject ANTH.
Scanned courses for subject APAN.
Scanned courses for subject APMA.
Scanned courses for subject APPH.
Scanned courses for subject APAM.
Scanned courses for subject ARCH.
Scanned courses for subject AHIS.
Scanned courses for subject ASCE.
Scanned courses for subject ASCM.
Scanned courses for subject AHUM.
Scanned courses for subject AHMM.
Scanned courses for subject ASST.
Scanned courses for subject ASTR.
Scanned courses for subject ASPH.
Scanned courses for subject ACLS.
Scanned courses for subject BHSC.
Scanned courses for subject BENG.
Scanned courses for subject BERL.
Scanned courses for subject BCHM.
Scanned course

Scanned courses for subject SDEV.
Scanned courses for subject SWHL.
Scanned courses for subject TMGT.
Scanned courses for subject THTR.
Scanned courses for subject THEA.
Scanned courses for subject TIBT.
Scanned courses for subject UKRN.
Scanned courses for subject URBS.
Scanned courses for subject UROL.
Scanned courses for subject UTCE.
Scanned courses for subject VIET.
Scanned courses for subject WLOF.
Scanned courses for subject WMST.
Scanned courses for subject YIDD.


In [86]:
locations = defaultdict(int)
ignored = {
    'ONLINE ONLY': [],
    'To be announced': [],
    'undefined': [],
    'OTHR OTHER': [],
}

for s in courses:
    loc = s.get('Location', 'undefined')
    if loc in ignored:
        ignored[loc].append(s)
    else:
        locations[s['Location']] += 1
                
ignoredCourses = sum(ignored.values(), []) # sums values into an initially empty list
print(f'Found {len(locations)} locations, but forced to ignore {len(ignoredCourses)}.')

Found 520 locations, but forced to ignore 4108.


In [88]:
sortedLocs = sorted(locations.items(), key=lambda kv: kv[1], reverse=True)
sortedLocs

[('5TH FLR Pupin Laboratories', 36),
 ('301M Fayerweather', 29),
 ('302 Fayerweather', 28),
 ('317 Hamilton Hall', 27),
 ('311 Fayerweather', 27),
 ('467 EXT Schermerhorn Hall [SCH]', 24),
 ('320 Havemeyer Hall', 24),
 ('402B International Affairs Building', 24),
 ('801 International Affairs Building', 24),
 ('401 Hamilton Hall', 23),
 ('501 International Affairs Building', 23),
 ('304 Hamilton Hall', 23),
 ('405A International Affairs Building', 23),
 ('1127 Seeley W. Mudd Building', 22),
 ('402 International Affairs Building', 22),
 ('407 International Affairs Building', 22),
 ('207 Mathematics Building', 21),
 ('833 Seeley W. Mudd Building', 21),
 ('313 Fayerweather', 21),
 ('200 S Fayerweather', 21),
 ('633 Seeley W. Mudd Building', 21),
 ('501A International Affairs Building', 21),
 ('410 International Affairs Building', 21),
 ('303 Seeley W. Mudd Building', 21),
 ('404 International Affairs Building', 21),
 ('823 International Affairs Building', 21),
 ('418 International Affairs 

Why are there so many ignored courses?

In [89]:
ignoredCourses = defaultdict(int)
for c in ignored['To be announced']:
    ignoredCourses[c['course']] += 1
sortedCourses = sorted(ignoredCourses.items(), key=lambda kv: kv[1], reverse=True)
sortedCourses

[('PHYSICAL EDUCATION ACTIVITIES', 58),
 ('INTEGR OF SCI & PRACT II', 17),
 ('LEADERSHIP DEVELOPMENT', 17),
 ('SEM-NEGOTIATION WORKSHOP', 8),
 ('HUMAN BEHAV/SOC ENVIRONMNT B', 8),
 ('CLIN PRACTCE WITH POP/PROF ISS', 8),
 ('LEGAL METHODS II', 6),
 ('SEMINAR: PUBLIC HEALTH PROGRAM PLANNING', 6),
 ('SOCIAL WORK PRACTICE II', 5),
 ('CAPSTONE CONSULTING SEMINAR', 4),
 ('PIVOT', 4),
 ('CRIMINAL LAW', 4),
 ('LAB:SURVEY DSGN & DATA COLLECTION', 4),
 ('ADVOCACY IN SW PRACTICE', 4),
 ('DESIGN & CONDUCT OF OBS EPI', 3),
 ('CONSULTING WORKSHOP', 3),
 ('PROPERTY (FOUNDATION)', 3),
 ('CONSTITUTIONAL LAW', 3),
 ('LEGISLATION AND REGULATION', 3),
 ('PROFESSIONAL RESPONSIBILITY', 3),
 ('S TRIAL PRACTICE', 3),
 ('EXPERIMENTAL SOCIAL WORK COURSES', 3),
 ('SOCIAL ENTERPRISE ADMIN', 3),
 ('RELATIONAL DATABASES AND SQL PROGRAMMING', 2),
 ('ALTERNATIVE ENERGY RESOURCES', 2),
 ('SEMINAR IN SEISMOLOGY', 2),
 ('PUBLIC HEALTH GIS', 2),
 ('HLTH CARE ACCOUNTING&BUDGETING', 2),
 ('HEALTHCARE MARKETING', 2),
 ('HEAL

Physical education was an obvious result, but the rest of the distribution doesn't say much. Let's look at subjects.

In [84]:
ignoredSubjects = defaultdict(int)
for c in ignored['To be announced']:
    ignoredSubjects[c['subject']] += 1
sortedSubjects = sorted(ignoredSubjects.items(), key=lambda kv: kv[1], reverse=True)
sortedSubjects

[('LAW_', 214),
 ('PHED', 58),
 ('SOCW', 51),
 ('HPMN', 48),
 ('POPF', 44),
 ('PUBH', 35),
 ('SOSC', 35),
 ('EPID', 27),
 ('BIST', 24),
 ('EHSC', 20),
 ('NURS', 20),
 ('PSCA', 14),
 ('THEA', 14),
 ('ENVB', 12),
 ('EESC', 11),
 ('ARCH', 6),
 ('HIFO', 4),
 ('NBHV', 4),
 ('SOAR', 4),
 ('EAEE', 3),
 ('GEND', 3),
 ('IRSH', 3),
 ('BMEN', 2),
 ('ENGL', 2),
 ('FILI', 2),
 ('FINC', 2),
 ('KREY', 2),
 ('HILI', 2),
 ('INAF', 2),
 ('MGMT', 2),
 ('SIPA', 2),
 ('WMST', 2),
 ('ACTU', 1),
 ('AKAD', 1),
 ('ANTH', 1),
 ('BUSI', 1),
 ('CANT', 1),
 ('CMBS', 1),
 ('CHEN', 1),
 ('CIEN', 1),
 ('COMM', 1),
 ('COMS', 1),
 ('DNCE', 1),
 ('DROM', 1),
 ('EAAS', 1),
 ('ECON', 1),
 ('EGYP', 1),
 ('ELEN', 1),
 ('EXEC', 1),
 ('HIST', 1),
 ('IEOR', 1),
 ('IKNS', 1),
 ('LWPS', 1),
 ('MRKT', 1),
 ('MSAE', 1),
 ('MDES', 1),
 ('MUSI', 1),
 ('OHMA', 1),
 ('PHIL', 1),
 ('PSYC', 1),
 ('RELI', 1),
 ('SINH', 1),
 ('THTR', 1),
 ('YIDD', 1)]

In [90]:
ignoredSubjects = defaultdict(int)
for c in ignored['undefined']:
    ignoredSubjects[c['subject']] += 1
sortedSubjects = sorted(ignoredSubjects.items(), key=lambda kv: kv[1], reverse=True)
sortedSubjects

[('BUSI', 326),
 ('LAW_', 171),
 ('BMEN', 158),
 ('SOCW', 148),
 ('MECE', 143),
 ('APAM', 142),
 ('ELEN', 140),
 ('ANTH', 114),
 ('AHIS', 104),
 ('PHIL', 102),
 ('CIEN', 98),
 ('IEOR', 97),
 ('SOCI', 97),
 ('NURS', 96),
 ('MSAE', 83),
 ('CHEN', 58),
 ('ARCH', 55),
 ('MUSI', 55),
 ('PSYC', 54),
 ('BINF', 49),
 ('OCCT', 47),
 ('RELI', 46),
 ('EAEE', 45),
 ('PHED', 33),
 ('CHEM', 30),
 ('ANCS', 29),
 ('MGMT', 27),
 ('PHYT', 26),
 ('APMA', 25),
 ('FINC', 24),
 ('EAAS', 22),
 ('HIST', 22),
 ('POLS', 22),
 ('CLPH', 21),
 ('EEEB', 20),
 ('ECON', 19),
 ('MEDI', 18),
 ('ENGL', 17),
 ('LATN', 17),
 ('APPH', 16),
 ('DROM', 16),
 ('INTC', 16),
 ('COMS', 15),
 ('GREK', 15),
 ('AFAS', 14),
 ('BIOL', 14),
 ('MATH', 13),
 ('ENGI', 12),
 ('EPID', 12),
 ('FILM', 12),
 ('RSRH', 12),
 ('SPAN', 12),
 ('THEA', 12),
 ('ADDN', 11),
 ('POPF', 11),
 ('MRKT', 10),
 ('FREN', 9),
 ('RESI', 9),
 ('THTR', 9),
 ('EHPS', 8),
 ('HPMN', 8),
 ('NBHV', 8),
 ('NEUR', 8),
 ('ORTS', 8),
 ('PSCA', 8),
 ('QMSS', 8),
 ('EESC', 